In [ ]:
import pandas as pd
import numpy as np
from typing import Dict, Any, List, Union, Optional
import logging

In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [ ]:
NON_ADJOURNABLE = [
    'Taxation and Issuance of Certificates',
    'Orders',
    'Appointments of  Mediator',
    'Screening of files for Mediation',
    'Post-judgment',
    'Re-activation',
    'Reactivation',
    'Notice of Taxation',
    'Entering Interlocutory Judgments',
    'Approval by DR', 
    'Registration/Filing-Application', 
    'Registration/Filing', 
    'Registration/Filing-Application',
 ]
RESOLVED_OUTCOMES = [
    'Ruling Delivered- Case Closed', 'Grant Confirmed', 'Matter Withdrawn',
    'Dismissed For Want Of Prosecution - Case Closed', 'Dismissed',
    'Terminated/ Struck Out/ Dismissed/case Closed', 'Judgment Delivered- Case Closed',
    'Application Allowed - Case Closed',
    'Matter Settled- Case Closed', 'Consent Recorded - Case Closed',
    'Judgment Delivered', 'Judgment Delivered- Acquittal',
    'Judgment Delivered- Convicted', 'Application Withdrawn - Case Closed',
    'Struck Out', 'Application Dismissed - Case Closed',
    'Out Of Court Settlement Reached', 'Terminated',
    'Ruling Delivered- Application Closed', 'Consolidated- Case Closed',
    'Interlocutory Judgement Entered', 'Abated', 'Limited Grant Issued',
    'Grant Revoked', 'Placed In Probation', 'Ruling Delivered- Accused Discharged',
    'Revision Declined', 'Retrial', 'Probation Orders Issued',
    'Matter Settled Through Mediation', 'Appeal Dismissed', 'Appeal Rejected', 
    'Order Issued - Case Closed',
    'Terminated',
    'Judgment Delivered- Aquittal',
    'Taxation',
    'Case Transferred To Nyeri High Court',
    'Sentenced',
    'Taxation Date Set',
    'Dismissed (Under Order 17 Rule 2)',
    'Matter Settled',
    'File Transfered -Case Closed',
    'Allowed',
    'Application Allowed',
    'Application Dismissed',
    'Application Withdrawn',
    'Case Closed',
    'Grant Issued',

]
CRIMINAL_CASES = [
    'Murder Case',
    'Criminal Revision',
    'Criminal Appeal',
    'Murder - Gender Justice Criminal Case',
    'Criminal Court Martial Appeal',
    'Anti-Corruption and Economic Crimes Revision',
    'Criminal Miscellaneous Application',
    'Criminal Applications', 
    'COA Criminal Appeal'
]
BROAD_CASE_TYPES = {
    'Civil Matter': [
        'Civil Suit',
        'Anti-Corruption and Economic Crimes Suit',
        'Family Originating Summons',
        'Family Civil Case',
        'HCC(OS) Family',
        'Commercial Admiralty',
        'Commercial Matters',
    ],
    
    'Adoption': 'Family Adoption',
    'Divorce': 'Family Divorce Cause',

    'Criminal Application': 'Criminal Miscellaneous Application',

    'Civil Application': [
        'Civil Case Miscellaneous',
        'Judicial Review Miscellaneous',
        'JR  Petition Miscellaneous',
        'Anti-Corruption and Economic Crimes Miscellaneous',
        'Commercial Miscellaneous',
        'Constitution and Human Rights Petitions Miscellaneous',
        'Family Miscellaneous',
        'Commercial Arbitration',
    ],
    'Judicial Review': [
        'Anti-Corruption and Economic Crime Judicial review',
        'Judicial Review ELC',
        'Judicial Review',
    ],
    'Criminal Revision': [
        'Criminal Revision',
        'Anti-Corruption and Economic Crimes Revision',
    ],
    'Criminal Appeal': [
        'Criminal Appeal',
        'Criminal Court Martial Appeal',
        'Anti-Corruption and Economic Crimes Appeal',
    ],
    'Civil Appeal': [
        'Family Appeal',
        'Civil Appeal',
        'Commercial Appeal',
        'Constitution and Human Rights Election Petition Appeal',
        'Constitution and Human Rights Petition Appeal',
        'Constitution and Human Rights Election Petition Appeal',
        'Gender Justice Civil Appeal',
        'Constitution and Human Rights Miscellaneous Election Petition Appeal (MEPA)',
    ],
    'Constitution Petition': [
        'Anti Corruption and Economic Crimes Petition',
        'High Court Criminal Petition',
        'Constitution and Human Rights Petition (Civil)',
        'Constitution and Human Rights Election Petition',
        'High Court Constitution and Human Rights Petitions (Criminal)',
        'Commercial Petition',
    ],
    'Probate Administration': [
        'Family P&A Intestate',
        'Family P&A Ad Litem',
        'Family P&A Ad Colligenda',
        'Family P&A Citation',
        'Family P&A Testate',
        'Family P&A Resealing of Grant',
        'Family P&A De Bonis Non',
        'Resealing of Grant',
        'Citation-Family',
    ],
    'Murder': [
        'Murder Case',
        'Murder - Gender Justice Criminal Case',
    ],
    'Tax Appeal': [
        'Commercial Income Tax Appeal',
        'Commercial Custom Tax Appeal',
    ],
    
    'Bankruptcy and Insolvency' : [
        'Commercial Insolvency Notice Petition',
        'Commercial Insolvency Petition',
        'Commercial Bankruptcy Notice',
        'Commercial Insolvency Cause',
        'Commercial Insolvency Notice',
        'Commercial Bankruptcy Cause',
        'Commercial Winding Up Cause',
    ]
}

MERIT_CATEGORY ={
    'Merit Resolution' : [    
    'Ruling Delivered- Case Closed', 
    'Judgment Delivered- Case Closed',
    'Judgment Delivered', 
    'Judgment Delivered- Acquittal',
    'Judgment Delivered- Convicted',
    'Ruling Delivered- Application Closed',
    'Retrial',
    'Appeal Dismissed',
    'Grant Revoked',
],
'Non Merit Resolution': [
    'Grant Confirmed', 
    'Matter Withdrawn',
    'Dismissed For Want Of Prosecution - Case Closed',
    'Dismissed',
    'Terminated/ Struck Out/ Dismissed/case Closed', 
    'Application Allowed - Case Closed',
    'Matter Settled- Case Closed', 
    'Consent Recorded - Case Closed',
    'Application Withdrawn - Case Closed',
    'Struck Out', 
    'Application Dismissed - Case Closed',
    'Out Of Court Settlement Reached', 
    'Terminated',
    'Consolidated- Case Closed',
    'Interlocutory Judgement Entered', 
    'Abated', 
    'Limited Grant Issued',
    'Placed In Probation', 
    'Revision Declined',  
    'Probation Orders Issued',
    'Matter Settled Through Mediation', 
    'Appeal Rejected', 
    'Order Issued - Case Closed',
    'Terminated',
    'Ruling Delivered- Accused Discharged',
    'File Transfered -case Closed', 
        'File Transferred',
    ]
} 


PRODUCTIVITY = {
    'Judgment Delivered': [
        'Judgment Delivered- Case Closed',
        'Judgment Delivered',
        'Judgment Delivered- Acquittal',
        'Judgment Delivered- Convicted',
        'Grant Revoked',
        'Retrial'
        ],
    'Ruling Case Closed': [
        'Ruling Delivered- Case Closed', 
        'Ruling Delivered- Accused Discharged',
        ],
    'Final Grant': [
        'Grant Confirmed',
        'Limited Grant Issued',
        ],
    'Case Withdrawn': [
        'Matter Withdrawn',
        'Application Withdrawn - Case Closed',
        ],
   'Out Of Court Settlement': [
        'Consent Recorded - Case Closed',
        'Matter Settled Through Mediation',
        'Out Of Court Settlement Reached',
    ],
    'Dismissed':[
        'Dismissed For Want Of Prosecution - Case Closed',
        'Dismissed',
        'Appeal Dismissed',
        'Terminated',
        'Terminated/ Struck Out/ Dismissed/case Closed',
    ],
    'Case Transfered': [
        'File Transfered -case Closed',
        'File Transferred',
    ],
    'Case Closed': [
        'Struck Out',
        'Application Dismissed - Case Closed',
        'Application Allowed - Case Closed',
        'Matter Settled- Case Closed',
        'Ruling Delivered- Application Closed',
        'Consolidated- Case Closed',
        'Abated',
        'Placed In Probation',
        'Revision Declined',
        'Probation Orders Issued',
        'Appeal Rejected',
        'Interlocutory Judgement Entered',
    ],
}

HC_PMMU_TIME_LINES = {
    'Murder': 360, 
    'Constitution Petition': 360, 
    'Criminal Revision': 180, 
    'Judicial Review': 360, 
    'Civil Matter': 360,
    'Anti Corruption': 360,
    'Income Tax Appeal': 180,
    'Matrimonial Property': 360,
    'Succession': 180,
}

TRANSFERED_CASES = ['File Transfered -case Closed', 
        'File Transferred',]

MERIT_OUTCOMES = [
    'Ruling Delivered- Case Closed', 
    'Judgment Delivered- Case Closed',
    'Judgment Delivered',
    'Judgment Delivered- Acquittal',
    'Judgment Delivered- Convicted',
    'Grant Revoked',
    'Ruling Delivered- Accused Discharged',
    'Retrial'
]


In [ ]:
def drop_nan_columns(df: pd.DataFrame, columns: List[str]) -> pd.DataFrame:
    """
    Drop rows containing NaN values from the specified columns of a DataFrame.

    Args:
        df (pd.DataFrame): The input DataFrame to process.
        columns (List[str]): A list of column names to check for NaN values.

    Returns:
        pd.DataFrame: The updated DataFrame with NaN-containing rows dropped.

    Raises:
        ValueError: If any of the specified columns are not present in the DataFrame.
    """
    # Validate that all specified columns exist in the DataFrame
    missing_columns = set(columns) - set(df.columns)
    if missing_columns:
        raise ValueError(f"Columns not found in DataFrame: {', '.join(missing_columns)}")

    # Identify columns with NaN values
    nan_columns = df[columns].columns[df[columns].isna().any()].tolist()

    # Log dropped rows if any
    if nan_columns:
        nan_count = df[columns].isna().sum()
        logger.info("Dropping rows with NaN values:")
        for col in nan_columns:
            logger.info(f"  {col}: {nan_count[col]} rows")

    # Drop rows with NaN values in specified columns
    original_row_count = len(df)
    df_cleaned = df.dropna(subset=columns)
    dropped_row_count = original_row_count - len(df_cleaned)

    if dropped_row_count > 0:
        logger.info(f"Total rows dropped: {dropped_row_count}")
    else:
        logger.info("No rows were dropped.")

    return df_cleaned

In [ ]:
def remove_duplicates(data: pd.DataFrame) -> pd.DataFrame:
    """
    Remove duplicates from a DataFrame.
    
    Args:
        data (pd.DataFrame): Input DataFrame.
        
    Returns:
        pd.DataFrame: DataFrame with duplicates removed.
    """
    num_duplicates = data.duplicated().sum()
    
    if num_duplicates > 0:
        logging.info(f"{num_duplicates} duplicates found.")
        data = data.drop_duplicates(keep="first").reset_index(drop=True)
        logging.info(f"{num_duplicates} duplicates removed.")
    else:
        logging.info("No duplicates found.")
    
    return data

In [ ]:

def create_date_column(df: pd.DataFrame, column_names: List[str], new_col: str) -> pd.DataFrame:
    """
    Creates a new date column in the DataFrame by concatenating the values of three specified columns.

    Args:
        df (pd.DataFrame): The DataFrame containing the data.
        column_names (List[str]): A list of three column names to be concatenated [year, month, day].
        new_col (str): The import commandsname of the new date column to be created.

    Returns:
        pd.DataFrame: The DataFrame with the new date column added.

    Raises:
        ValueError: If the input list doesn't contain exactly three column names or if columns are missing.
    """
    if len(column_names) != 3:
        raise ValueError("column_names must contain exactly three elements: [year, month, day]")

    year_col, month_col, day_col = column_names

    # Check if all required columns exist in the DataFrame
    missing_columns = set(column_names) - set(df.columns)
    if missing_columns:
        raise ValueError(f"Missing columns in DataFrame: {', '.join(missing_columns)}")

    # Create copies to avoid SettingWithCopyWarning
    df = df.copy()

    try:
        # Convert year and day columns to integers
        df[year_col] = df[year_col].astype(float).astype(int)
        df[day_col] = df[day_col].astype(float).astype(int)

        # Concatenate the columns to create a date string
        df[new_col] = (df[year_col].astype(str) + '-' + 
                       df[month_col].astype(str) + '-' + 
                       df[day_col].astype(str))

        # Convert to datetime
        df[new_col] = pd.to_datetime(df[new_col], errors='coerce')

        # Log information about the conversion
        valid_dates = df[new_col].notna().sum()
        logger.info(f"Created new date column '{new_col}'. Valid dates: {valid_dates}/{len(df)}")

    except Exception as e:
        logger.error(f"Error creating date column: {str(e)}")
        raise

    return df

In [ ]:
def strip_dataframe_columns(df):
    """Strips leading and trailing whitespace from all columns in a Pandas DataFrame.

    Args:
        df (pandas.DataFrame): The DataFrame to modify.

    Returns:
        pandas.DataFrame: The modified DataFrame with stripped columns.
    """

    try:
        df = df.astype(str).apply(lambda x: x.str.strip())
        logger.info("str.strip() applied successfully to all columns.")
        return df
    except Exception as e:
        logger.error(f"Error applying str.strip(): {e}")
        return None

In [ ]:
def apply_title_case(text):
    """
    Apply title case to a given string.
    
    Args:
        text: The input string to process.
    
    Returns:
        str: The processed string in title case.
    """
    if pd.isna(text):
        return np.nan
    if not isinstance(text, str):
        logger.warning(f"Non-string value encountered: {text}")
        return str(text)
    return text.title()

In [ ]:

def add_title_case(df: pd.DataFrame, column: str) -> pd.DataFrame:
    """
    Process the 'outcome' column of the DataFrame by applying title case.
    
    Args:
        df (pd.DataFrame): The input DataFrame containing the 'outcome' column.
    
    Returns:
        pd.DataFrame: The DataFrame with the processed 'outcome' column.
    """
    if 'outcome' not in df.columns:
        logger.error("'outcome' column not found in the DataFrame")
        return df

    original_null_count = df[column].isnull().sum()
    
    df['outcome'] = df[column].apply(apply_title_case)
    
    new_null_count = df[column].isnull().sum()
    if new_null_count > original_null_count:
        logger.warning(f"Number of null values in 'outcome' increased from {original_null_count} to {new_null_count}")
    
    non_string_count = df[column].apply(lambda x: not isinstance(x, str) if pd.notna(x) else False).sum()
    if non_string_count > 0:
        logger.warning(f"Found {non_string_count} non-string values in 'outcome' after processing")

    return df

In [ ]:
def drop_null_values(df: pd.DataFrame, column_name: str = 'outcome') -> pd.DataFrame:
    """
    Drop rows from the DataFrame where the specified column contains null values.

    Args:
        df (pd.DataFrame): The DataFrame from which to drop rows.
        column_name (str): The name of the column to check for null values. Default is 'outcome'.

    Returns:
        pd.DataFrame: The DataFrame with rows containing null values in the specified column dropped.
    """
    initial_row_count: int = df.shape[0]
    cleaned_df: pd.DataFrame = df.dropna(subset=[column_name])
    final_row_count: int = cleaned_df.shape[0]
    dropped_row_count: int = initial_row_count - final_row_count
    
    if dropped_row_count > 0:
        logger.info(f"Total dropped rows with null values in '{column_name}': {dropped_row_count}")
    else:
        logger.info(f"No rows dropped with null values in '{column_name}'")
    return cleaned_df

In [ ]:
def categorize_case(case_type: str, criminal_cases: Optional[List[str]]) -> str:
    """
    Categorize a case as 'Criminal' or 'Civil' based on its type.
    
    Args:
        case_type (str): The type of the case.
        criminal_cases (Optional[List[str]]): List of case types considered as criminal.
        
    Returns:
        str: 'Criminal' if the case type is in the criminal cases list or if criminal_cases is None, 'Civil' otherwise.
    """
    if criminal_cases is None:
        return 'Criminal'
    else:
        return 'Criminal' if case_type in criminal_cases else 'Civil'

In [ ]:
def categorize_cases(df: pd.DataFrame, criminal_cases: Optional[List[str]] = None) -> pd.DataFrame:
    """
    Categorize all cases in the DataFrame as 'Criminal' or 'Civil'.
    
    Args:
        df (pd.DataFrame): The input DataFrame containing case data.
        criminal_cases (Optional[List[str]]): List of case types considered as criminal.
            If None, all cases are categorized as 'Criminal'.
        
    Returns:
        pd.DataFrame: DataFrame with an added 'nature' column indicating case nature.
    """
    df['nature'] = df['case_type'].apply(lambda x: categorize_case(x, criminal_cases))

    # Check for presence of both case types
    if 'Criminal' not in df['nature'].values:
        logging.warning("No criminal cases found in the DataFrame.")
    if 'Civil' not in df['nature'].values:
        logging.warning("No civil cases found in the DataFrame.")
    
    return df

In [ ]:
def apply_dict(value: Any, dictionary: Dict[str, Union[str, List[Any]]]) -> Union[str, None]:
    """
    Find all keys in a dictionary where the given value matches.

    Args:
        value: The value to search for.
        dictionary: The dictionary to search in.

    Returns:
        A list of keys where the value matches, or None if no matches are found.
    """
    matching_keys = []
    for key, dict_value in dictionary.items():
        if isinstance(dict_value, str) and dict_value == value:
            matching_keys.append(key)
        elif isinstance(dict_value, list) and value in dict_value:
            matching_keys.append(key)
    
    if not matching_keys:
        return None
    elif len(matching_keys) == 1:
        return matching_keys[0]
    else:
        return matching_keys

In [ ]:
def generate_case_num(df: pd.DataFrame, caseid_type_col: str, caseid_no_col: str, filed_yyyy_col: str, new_col='case_number') -> pd.DataFrame:
    """
    Generates a case number by concatenating court, caseid_type, caseid_no, and filed_yyyy columns.

    Args:
        df (pd.DataFrame): The dataframe containing the necessary columns.
        court_col (str): The name of the column containing court information.
        caseid_type_col (str): The name of the column containing case ID type.
        caseid_no_col (str): The name of the column containing case ID number.
        filed_yyyy_col (str): The name of the column containing the year the case was filed.
        new_col (str): The name of the new column to be created for the case number. Default is 'case_num'.

    Returns:
        pd.DataFrame: DataFrame with the new case number column.
    """
    df[new_col] = df[caseid_type_col] + '/' + df[caseid_no_col] + '/' + df[filed_yyyy_col].astype(str)
    return df

def is_concluded(outcome: str, resolved_outcomes: List[str]) -> int:
    """
    Determine if the case is concluded based on the outcome.
    
    Args:
        outcome (str): The outcome of the case.
        resolved_outcomes (List[str]): List of outcomes considered as resolved.
        
    Returns:
        int: 1 if the case outcome is resolved, 0 otherwise.
    """
    return 1 if outcome in resolved_outcomes else 0

In [ ]:
def is_case_registered(outcome: str, activity_date: Union[pd.Timestamp, str], filed_date: Union[pd.Timestamp, str]) -> bool:
    """
    Determine if a case is registered based on its outcome and dates.

    Args:
        outcome (str): The outcome of the case.
        activity_date (Union[pd.Timestamp, str]): The date of the activity.
        filed_date (Union[pd.Timestamp, str]): The date the case was filed.

    Returns:
        bool: True if the case is registered, False otherwise.
    """
    try:
        # Normalize the outcome string
        normalized_outcome = outcome.strip().lower()

        # Check if the outcome indicates registration
        is_registered_outcome = 'case registered/filed' in normalized_outcome 

        # Convert dates to pd.Timestamp if they're strings
        if isinstance(activity_date, str):
            activity_date = pd.to_datetime(activity_date, errors='coerce')
        if isinstance(filed_date, str):
            filed_date = pd.to_datetime(filed_date, errors='coerce')

        # Check if dates are equal
        dates_match = pd.notna(activity_date) and pd.notna(filed_date) and activity_date == filed_date

        is_registered = is_registered_outcome and dates_match

        if is_registered:
            logger.debug(f"Case registered: outcome='{outcome}', activity_date={activity_date}, filed_date={filed_date}")
        
        return is_registered

    except Exception as e:
        logger.error(f"Error in is_case_registered: {e}")
        return False

In [ ]:
def process_case_status(df: pd.DataFrame, resolved_outcome: List[str]) -> pd.DataFrame:
    """
    Process the DataFrame to add 'concluded' and 'registered' columns.

    Args:
        df (pd.DataFrame): The input DataFrame containing case information.

    Returns:
        pd.DataFrame: The DataFrame with added 'concluded' and 'registered' columns.

    Raises:
        ValueError: If required columns are missing from the DataFrame.
    """
    required_columns = ['outcome', 'activity_date', 'filed_date']
    missing_columns = set(required_columns) - set(df.columns)
    if missing_columns:
        raise ValueError(f"Missing required columns: {', '.join(missing_columns)}")

    #df['concluded'] = df['outcome'].apply(is_case_concluded)
    df['concluded'] = df['outcome'].apply(lambda x: is_concluded(x, resolved_outcome))
    df['registered'] = df.apply(lambda row: is_case_registered(row['outcome'], row['activity_date'], row['filed_date']), axis=1)

    logger.info(f"Processed {len(df)} cases")
    logger.info(f"Concluded cases: {df['concluded'].sum()}")
    logger.info(f"Registered cases: {df['registered'].sum()}")

    return df

In [ ]:
def analyze_court_outcomes(df: pd.DataFrame, start_date: str, end_date: str, outcome: str) -> pd.DataFrame:
    """
    Calculate the number of case outcomes per court within a specified period.
    
    Args:
        df (pd.DataFrame): A pandas DataFrame containing the data.
        start_date (str): The starting date of the period (YYYY-MM-DD format).
        end_date (str): The ending date of the period (YYYY-MM-DD format).
        outcome (str): A column representing the outcome of interest.
        
    Returns:
        pd.DataFrame: A DataFrame showing the number of resolved cases per court and case category.
    """
    try:
        period_start = pd.to_datetime(start_date)
        period_end = pd.to_datetime(end_date)
        
        if period_start > period_end:
            raise ValueError("start_date must be earlier than end_date")
        
        required_columns = {'court', 'broad_case_type', 'activity_date', outcome}
        if not required_columns.issubset(df.columns):
            missing_columns = required_columns - set(df.columns)
            raise KeyError(f"Missing required columns: {missing_columns}")
        
        filtered_cases = df[
            (df['activity_date'] >= period_start) &
            (df['activity_date'] <= period_end) &
            (df[outcome] == 1)
        ]
        
        if filtered_cases.empty:
            logging.warning("No cases found for the given date range and outcome.")
   
        outcome_by_type = (
            filtered_cases
            .groupby(['court', 'broad_case_type'])
            .size()
            .reset_index(name='num_cases')
        )

        result = outcome_by_type.pivot_table(
            index='court', 
            columns='broad_case_type', 
            values='num_cases', 
            fill_value=0
        )
        
        logging.info("Successfully calculated case outcomes per court.")
        return result
    
    except Exception as e:
        logging.error(f"An error occurred: {e}")
        raise

In [ ]:
def check_time_limit(age, case_category, concluded, time_lines):
    """
    Check if a case falls within the specified time limit for its category and is concluded.
    
    Parameters:
        age (int): The age of the case in days.
        case_category (str): The category of the case.
        concluded (int): The status of the case conclusion (1 for concluded, 0 otherwise).
        time_lines (dict): A dictionary with case categories as keys and time limits as values.
        
    Returns:
        int: 1 if the case is within the time limit and concluded, otherwise 0.
    """
    time_limit = time_lines.get(case_category, 0)
    return 1 if age <= time_limit and concluded == 1 else 0

### Analysis of multiple resolved outcomes

In [ ]:

# Define the priority order
priority_order = [
    'Judgment Delivered- Case Closed',
    'Judgment Delivered',
    'Judgment Delivered- Acquittal',
    'Judgment Delivered- Convicted',
    'Grant Revoked',
    'Ruling Delivered- Case Closed',
    'Ruling Delivered- Accused Discharged'
]

# Create a priority mapping for efficient lookups
priority_mapping = {value: index for index, value in enumerate(priority_order)}

def get_resolved_cases_by_court(df, outcome_col='outcome', court_col='court'):
    # Assign priority to each row based on the outcome
    df['priority'] = df[outcome_col].map(priority_mapping)
    
    # Filter rows with resolved cases
    resolved_cases = df[df['priority'].notna()]
    
    # Drop duplicates to ensure one outcome per case with the highest priority
    resolved_cases = resolved_cases.sort_values(by='priority').drop_duplicates(subset=['case_number', court_col], keep='first')
    
    # Drop the priority column as it is no longer needed
    resolved_cases = resolved_cases.drop(columns=['priority'])
    
    # Pivot the table
    pivot_table = resolved_cases.pivot_table(index=court_col, columns=outcome_col, values='case_number', aggfunc='count', fill_value=0)
    
    return pivot_table


In [ ]:
# Load data (modify file paths and separators as needed)
df_a = pd.read_csv("master.csv", sep='\t', names=["master"], header=0)
df_b = pd.read_csv("mixed.csv", sep='\t', names=["mixed"], header=0)
import re

def tokenize_name(name):
    """
    Lowercases the name, removes non-alphanumeric characters except spaces,
    and returns a frozenset of tokens (words).
    """
    # Remove punctuation except spaces
    name_clean = re.sub(r'[^\w\s]', '', name.lower())
    # Split on whitespace
    tokens = name_clean.split()
    # Convert list to a frozenset so order doesn't matter
    return frozenset(tokens)
# --------------------------------------------------------------------
# 1. Read your CSV. If it's truly tab-delimited, use sep='\t'.
#    Otherwise, adjust the separator accordingly.
# --------------------------------------------------------------------
judges_df = pd.read_csv(f'{file_path}/Judges.csv', header=0)
# Drop empty values
judges_df = judges_df.dropna(subset=["master"])
df = df.dropna(subset=["judge_1"])
# Create a set of tokenized master names
master_tokens = set(judges_df["master"].apply(tokenize_name))
# Filter df_b to keep only rows where the mixed name matches a master name
df_filtered = df[df["judge_1"].apply(lambda x: tokenize_name(x) in master_tokens)]
# Get unique names of judge_1 

judges_name = df_filtered['judge_1'].unique().tolist()
judges = pd.DataFrame(judges_name)
judges.to_csv(f'{file_path}/judges.csv')

# --------------------------------------------------------------------
# 2. Build a set of tokenized names from the 'master' column.
# --------------------------------------------------------------------
master_tokens = set(tokenize_name(m) for m in df["master"])

# --------------------------------------------------------------------
# 3. Filter rows where the 'mixed' name also appears in the master set
#    (by token comparison).
# --------------------------------------------------------------------
mask = df["mixed"].apply(lambda x: tokenize_name(x) in master_tokens)
df_filtered = df[mask]

# df_filtered now contains only the rows where the 'mixed' name
# matches a 'master' name (ignoring order/punctuation).
print(df_filtered)



import re
def clean_name(name):
    """Lowercase and remove extra punctuation/spaces."""
    return re.sub(r'[^\w\s]', '', name.lower()).strip()
from rapidfuzz import process, fuzz
master_names = judges_df["master"].unique().tolist()
master_cleaned = [clean_name(m) for m in master_names]

def is_match_fuzzy(mixed_name, threshold=57):
    """
    Return True if 'mixed_name' matches any name in 'master_names'
    with a fuzzy ratio >= threshold.
    """
    # Clean the name
    name_clean = clean_name(mixed_name)
    # Find best fuzzy match among the master names
    best_match = process.extractOne(name_clean, master_cleaned, scorer=fuzz.ratio)
    # best_match looks like (matched_string, score, index)
    if best_match and best_match[1] >= threshold:
        return True
    return False

df_filtered_fuzzy = df[df["judge_1"].apply(is_match_fuzzy)]
df_filtered_fuzzy.groupby('judge_1')['judge_1'].count()

In [ ]:
file_path = '/home/fiend/Documents'
raw_df = pd.read_csv(f'{file_path}/nyahururu.csv')
df = raw_df.set_axis(['caseid_type', 'case_type','caseid_no', 'activity_date'], axis=1)
df['activity_date'] = pd.to_datetime(df['activity_date'], errors='coerce')

In [ ]:
df = generate_case_num(raw_df, 'caseid_type', 'caseid_no', 'filed_yyyy')

In [ ]:
# drop row with missing values on raw_df['date_dd']
df = df.dropna(subset=['date_dd'])

In [ ]:
df['outcome'] = df['outcome'].str.strip() 

In [ ]:
df = add_title_case(df, 'outcome')

In [ ]:
df['outcome'] = df['outcome'].replace('Terminated/ Struck Out/ Dismissed/Case Closed', 'Terminated')
#cutoff_date = pd.Timestamp('2024-06-30') 
#raw_df = raw_df.rename(columns={'court_name': 'court'})
df = drop_nan_columns(df, ['date_dd', 'date_mon', 'date_yyyy'])
df = create_date_column(df.copy(), ['date_dd', 'date_mon', 'date_yyyy'], 'activity_date')
df = create_date_column(df.copy(), ['filed_dd', 'filed_mon', 'filed_yyyy'], 'filed_date')

In [ ]:
df = remove_duplicates(df)
df = drop_null_values(df)

In [ ]:
outcomes = df['outcome'].unique()

In [ ]:
df = strip_dataframe_columns(df)

In [ ]:
df = process_case_status(df, RESOLVED_OUTCOMES)

In [ ]:
# sort by case_number and activity_date
df = df.sort_values(by=['activity_date', 'case_number'])

In [ ]:
df['max_resolved'] = df.groupby('case_number')['concluded'].transform('max')
# convert max_resolved to int
df['max_resolved'] = df['max_resolved'].astype(int)
# keep if max_resolved is not 1 
pending_df = df[df['max_resolved'] != 1]

In [ ]:
pending_df = pending_df.drop_duplicates(subset=['case_number'], keep='last')

In [ ]:
pending_df['activity_date'] = pd.to_datetime(pending_df['activity_date'], errors='coerce')
pending_df['age'] = (pd.Timestamp.now() - pending_df['activity_date']).dt.days

In [ ]:
# set concluded to 1 if outcome == "Judgment Date Given" & age_case > 360
pending_df['concluded'] = np.where((pending_df['outcome'] == 'Judgment Date Given') & (pending_df['age'] > 360), 1, pending_df['concluded'])

In [ ]:
pending_df['max_resolved'] = pending_df.groupby('case_number')['concluded'].transform('max')
# convert max_resolved to int
pending_df['max_resolved'] = pending_df['max_resolved'].astype(int)
# keep if max_resolved is not 1 
pending_df = df[df['max_resolved'] != 1]

In [ ]:
df['age'] = (pd.Timestamp.now() - df['activity_date']).dt.days
df['age_category'] = pd.cut(
    df['age'], 
    bins=[0, 360, 1080, df['age'].max()], 
    labels=['0-360 days', '360-1080 days', 'Over 1080 days']
)


In [ ]:
df

In [ ]:
# create a pivot table of age category and case_type
df.pivot_table(index='case_type', columns='age_category', values='caseid_no', aggfunc='count', fill_value=0).to_csv(f'{file_path}/nyahururu_age_category.csv')

In [ ]:
raw_df.pivot_table(index='case_type', columns='age_category', values='case_number', aggfunc='count', fill_value=0).to_csv(f'{file_path}/age_category.csv')

In [ ]:
pending_df.to_csv(f'{file_path}/final_df.csv', index=False)